In [1]:
import nltk, math, codecs
from gensim.models import Doc2Vec
from nltk.cluster.kmeans import KMeansClusterer
import re
import pymorphy2
from datetime import datetime

fname = 'noStopLemma_PV-DBOW_wrd-vec_1it_2win_6mincount_alpha25-25_sz80.model'

model = Doc2Vec.load(fname)

morph = pymorphy2.MorphAnalyzer()

C:\Anaconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    txt = f.read().split('\n')
stw = set(txt)

In [3]:
#lemmatized test

start_time = datetime.now()
corpus = codecs.open('test_headlines_short.txt', mode="r", encoding="utf-8")
lines = corpus.read().lower().split('\r\n')
lemm_lines = []
for line in lines:
    fixed = ''.join([x if x.isalnum() or x.isspace() else " " for x in line ]).split()
    fixedNoStop = []
    for fix in fixed:
        if fix not in stw:
            fix = morph.parse(fix)[0].normal_form
            fixedNoStop.append(fix)
    lemm_lines.append(fixedNoStop)

    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:07.156179


In [ ]:
print(lines[:10])

In [3]:
NUM_CLUSTERS = 35 #25 #35 #25 #20 #40 #30 

def preprocess(str):
    # remove links
    str = re.sub(r'http(s)?:\/\/\S*? ', "", str)
    return str


def preprocess_document(text):
    #text = preprocess(text)
    fixedNoStop = []
    fixed = ''.join([x if x.isalnum() or x.isspace() else " " for x in text ]).split()
    for fix in fixed:
        if fix not in stw:
            fix = morph.parse(fix)[0].normal_form
            fixedNoStop.append(fix)
    return fixedNoStop

start_time = datetime.now()

#data = <sparse matrix that you would normally give to scikit>.toarray()

corpus = codecs.open('test_headlines_short.txt', mode="r", encoding="utf-8")
lines = corpus.read().lower().split('\r\n')
count = len(lines)

vectors = []

print("inferring vectors")
duplicate_dict = {}
used_lines = []
for i, t in enumerate(lines):
    if t not in duplicate_dict:#i % 2 == 0 and
        duplicate_dict[t] = True
        used_lines.append(t)
        vectors.append(model.infer_vector(preprocess_document(t)))

print("done")



kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=20)
assigned_clusters = kclusterer.cluster(vectors, assign_clusters=True)
print('Cluster assigning done!')
    
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

inferring vectors
done
Cluster assigning done!
Duration: 0:09:11.363663


In [18]:
len(used_lines)

2210

In [4]:
clustersizes = []

def distanceToCentroid():
    for i in range(0, NUM_CLUSTERS):
        clustersize = 0
        for j in range(0, len(assigned_clusters)):
            if (assigned_clusters[j] == i):
                clustersize+=1
        clustersizes.append(clustersize)
        dist = 0.0
        centr = kclusterer.means()[i]
        for j in range(0, len(assigned_clusters)):
            if (assigned_clusters[j] == i):
                dist += pow(nltk.cluster.util.cosine_distance(vectors[j], centr),2)/clustersize
        dist = math.sqrt(dist)
        print("distance cluster: "+str(i)+" RMSE: "+str(dist)+" clustersize: "+str(clustersize))

def nClosestToCentroid(cluster_id, n):
    #clustersize = len(get_titles_by_cluster(cluster_id))
    centr = kclusterer.means()[cluster_id]
    distances = []
    for j in range(0, len(assigned_clusters)):
        if (assigned_clusters[j] == cluster_id):
            distances.append((used_lines[j], nltk.cluster.util.cosine_distance(vectors[j], centr)))
    distances = sorted(distances, key=lambda tup: tup[1])
    return distances[:n]


def get_titles_by_cluster(id):
    list = []
    for x in range(0, len(assigned_clusters)):
        if (assigned_clusters[x] == id):
            list.append(used_lines[x])
    return list

def get_topics(titles):
    from collections import Counter
    words = [preprocess_document(x) for x in titles]
    words = [word for sublist in words for word in sublist]
    #filtered_words = [word for word in words if word not in stw]
    count = Counter(words)
    print(count.most_common()[:5])


def cluster_to_topics(id):
    get_topics(get_titles_by_cluster(id))

In [12]:
for clstr in range(NUM_CLUSTERS):
    cluster_to_topics(clstr)

[('теракт', 19), ('задержать', 15), ('мужчина', 9), ('террорист', 8), ('боевик', 8)]
[('рубль', 23), ('миллиард', 16), ('миллион', 12), ('доллар', 8), ('евро', 8)]
[('путин', 29), ('глава', 15), ('президент', 14), ('мид', 13), ('встреча', 13)]
[('россия', 16), ('рост', 8), ('считать', 8), ('россиянин', 7), ('рейтинг', 6)]
[('нефть', 14), ('рассказать', 9), ('добыча', 8), ('рынок', 7), ('роснефть', 6)]
[('выборы', 23), ('президент', 16), ('выбор', 13), ('франция', 11), ('макрон', 10)]
[('россия', 6), ('рассказать', 5), ('путин', 5), ('дом', 4), ('трамп', 3)]
[('каталония', 12), ('мадрид', 5), ('полиция', 5), ('призвать', 4), ('протест', 4)]
[('москва', 20), ('рейс', 8), ('пассажир', 7), ('метро', 7), ('станция', 6)]
[('заявить', 29), ('россия', 22), ('сша', 22), ('ес', 16), ('путин', 14)]
[('сми', 19), ('сообщить', 16), ('москва', 8), ('сообщение', 8), ('источник', 8)]
[('крым', 14), ('украина', 8), ('киев', 7), ('россия', 6), ('слово', 5)]
[('экс', 16), ('глава', 16), ('министр', 9), (

In [ ]:
get_titles_by_cluster(8)

In [ ]:
distanceToCentroid()

In [6]:
len(assigned_clusters)

2210

In [14]:
len(get_titles_by_cluster(5))

70

In [15]:
nClosestToCentroid(5, 23)

[('макрона во втором туре выборов во франции поддержат 61% избирателей, показал опрос',
  0.1524389442078623),
 ('"дух поражения" и "война против всех": как прошли дебаты макрона и ле пен',
  0.18256925746362251),
 ('ле пен обгонит макрона в первом туре выборов, показал опрос',
  0.18304331194618217),
 ('в киргизии лидера парламентской оппозиции выдвинули кандидатом в президенты',
  0.18750982616175227),
 ('в германии проходят парламентские выборы', 0.18943013591974256),
 ('народная партия лидирует на выборах в австрии', 0.19061378682680263),
 ('трамп поздравил макрона с победой на выборах во франции',
  0.19440853465870833),
 ('макрон представил программу на выборах президента франции',
  0.19690478219429131),
 ('блок меркель лидирует после подсчета голосов в 227 округах',
  0.19743194481812909),
 ('ле пен проголосовала на выборах президента. прямая трансляция',
  0.20134079074080091),
 ('врио ярославского губернатора лидирует на региональных выборах',
  0.20745874946175824),
 ('премь

In [8]:
#35 кластеров, 30 повторений
for clstr in range(NUM_CLUSTERS):
    n = round(len(get_titles_by_cluster(clstr))/3)
    print(nClosestToCentroid(clstr, n))
    print('####\n')

[('мэа повысило прогноз-2017 по добыче нефти в россии', 0.17539732269494024), ('недельная ставка roisfix в четверг выросла до 10,08%', 0.21044693093348765), ('цены на бензин выросли до рекордного уровня', 0.2135429281913499), ('набиуллина не увидела минусов в текущей низкой инфляции в россии', 0.21824571940956339), ('цены на нефть незначительно снизились после резкого роста', 0.22784902470079516), ('"победа" стала мировым лидером по темпам роста выручки среди лоукостеров', 0.23373268420004811), ('минэнерго в ближайшее время ожидает восстановления цен на нефть', 0.234335579845251), ('волков: урожайность зерновых в мордовии за 3 года выросла почти на треть', 0.23546724049247048), ('в глубокий минус. как схлопывается биткоин-пузырь', 0.23615770036936623), ('москва поднялась на 15 позиций в рейтинге самых дорогих городов мира', 0.23936330823115004), ('в минэкономразвития рассказали об ожидаемой инфляции в россии в сентябре', 0.25393009521425292), ('товарооборот россии и италии вырос на 28%

In [9]:
for clstr in range(NUM_CLUSTERS):
    cluster_to_topics(clstr)

[('россия', 9), ('рост', 8), ('нефть', 8), ('2017', 7), ('вырасти', 7)]
[('рубль', 22), ('миллиард', 15), ('миллион', 10), ('доллар', 6), ('1', 6)]
[('задержать', 17), ('теракт', 13), ('подозревать', 7), ('полицейский', 7), ('полиция', 7)]
[('назвать', 9), ('слово', 5), ('рассказать', 5), ('заявить', 5), ('путин', 4)]
[('рассказать', 5), ('россиянин', 5), ('российский', 5), ('россия', 5), ('трамп', 4)]
[('россия', 18), ('мид', 13), ('ес', 13), ('сша', 13), ('заявить', 12)]
[('рассказать', 9), ('роснефть', 5), ('система', 5), ('рынок', 5), ('нефть', 5)]
[('москва', 17), ('метро', 8), ('станция', 7), ('петербург', 6), ('звонок', 6)]
[('мужчина', 15), ('полицейский', 10), ('задержать', 9), ('ребёнок', 7), ('водитель', 5)]
[('боевик', 9), ('турция', 7), ('иго', 7), ('ирак', 7), ('заявить', 6)]
[('суд', 26), ('дело', 12), ('приговор', 7), ('решение', 6), ('арест', 5)]
[('министр', 13), ('отставка', 11), ('глава', 9), ('президент', 8), ('пост', 8)]
[('сша', 17), ('встреча', 13), ('мид', 13),

In [43]:
indxs = [6,7,12,16,20,21,24,30,33,34]
all_polemics = []
for indx in indxs:
    arr = []
    n = len(get_titles_by_cluster(indx))
    head_dists = nClosestToCentroid(indx, n)
    for head in head_dists:
        arr.append(head[0])
    all_polemics.append((indx, arr))

In [49]:
a=0
for topic in all_polemics:
    print(topic[0],len(topic[1]))

6 60
7 52
12 59
16 59
20 79
21 107
24 44
30 104
33 86
34 67


In [46]:
with open('polemic_headlines_by_themes.txt', 'w', encoding='utf-8') as f:
    for topic in all_polemics:
        f.write(str(topic[0])+'\n')
        for headline in topic[1]:
            f.write(headline+'\n')
        f.write('\n')